# Day 7

Link to challenge: https://adventofcode.com/2022/day/7

## Part 1

In this challenge we have to build a file tree from our puzzle input.

In [97]:
from anytree import Node, RenderTree, PreOrderIter
from typing import List
import re

root: Node = Node("/")
pwd: Node  = root

def parse_input(lines: List[str]):
    for line in lines:
        if line.startswith("$ cd "):
            execute_cd(line)
        elif line.startswith("dir "):
            append_dir_from_ls_result(line)
        elif is_file_line(line):
            append_file_from_ls_result(line)
            
def is_file_line(line: str):
    return re.match("\d.*", line)

def render_tree(tree):
    for pre, fill, node in RenderTree(tree):
        print("%s%s" % (pre, node.name))

def append_dir_from_ls_result(line: str):
    dirname = line.strip().split(" ")[1]
    dirnode = Node("d_" + dirname)
    dirnode.parent = pwd

def append_file_from_ls_result(line: str):
    filesize, filename = line.strip().split(" ")
    filenode = Node("f_" + str(filesize) + "_" + filename)
    filenode.parent = pwd

def get_node_size(node: Node): 
    if str(node.name).startswith("f_"):
        return int(str(node.name).split("_")[1])
    else:
        return sum(map(get_node_size, node.children))


def execute_cd(line: str):
    arg1 = line.strip().split(" ")[2]
    global pwd
    if arg1 == "/":
        pwd = root
    elif arg1 == "..":
        pwd = pwd.parent
    else :
        map(lambda c: c.name, pwd.children)
        for c in pwd.children:
            if c.name == "d_" + arg1:
                pwd = c
                return

# this implementation has a really bad asymptotic runtime complexity. For better runtime we could build a tree of sizes and work on that to prevent calculating the same thing multiple times
def sum_of_directories_smaller_than_x(root, x):
    return sum(filter(lambda size: size <= x, [get_node_size(node) for node in PreOrderIter(root) if str(node.name).startswith("d_")]))

# this implementation has a really bad asymptotic runtime complexity. For better runtime we could build a tree of sizes and work on that to prevent calculating the same thing multiple times
def smallest_directoy_larger_than_x(root, x):
    return min(filter(lambda size: size >= x, [get_node_size(node) for node in PreOrderIter(root) if str(node.name).startswith("d_")]))

In [98]:
with open("puzzle-input/7.txt", "r") as f:
    parse_input(list(map(str, f.readlines())))
    render_tree(root)
    print("Part 1", sum_of_directories_smaller_than_x(root, 100000))
    print("Part 2", smallest_directoy_larger_than_x(root, 30000000-(70000000-get_node_size(root))))

/
├── d_fnsvfbzt
│   └── f_62158_sfwnts.hbj
├── d_hqdssf
│   ├── f_45626_cvcbmcm
│   ├── d_dlsmjsbz
│   │   └── f_9205_qcqbgd.lzd
│   ├── d_hqdssf
│   │   ├── f_105963_mhqs.zrn
│   │   └── f_87909_slwshm.nwr
│   ├── d_mhqs
│   │   ├── d_ctfl
│   │   │   └── d_shzgg
│   │   │       ├── f_18097_cvcbmcm
│   │   │       ├── f_289064_mhqs
│   │   │       ├── f_208557_slwshm.nwr
│   │   │       ├── f_283449_vjw
│   │   │       └── d_wfbvtfmr
│   │   │           ├── f_263560_dssbpgnl.szh
│   │   │           ├── d_hnqjmq
│   │   │           │   └── f_3307_rjd.lgh
│   │   │           ├── f_76551_jvvl.rcs
│   │   │           ├── f_195911_lncqsmj
│   │   │           └── f_185776_slwshm.nwr
│   │   ├── f_45923_jvvl.rcs
│   │   ├── d_jzjm
│   │   │   └── f_31719_rjjrg.pjq
│   │   ├── d_lncqsmj
│   │   │   └── d_mhqs
│   │   │       └── f_138296_wfbvtfmr
│   │   ├── d_mhqs
│   │   │   ├── f_175499_jvvl.rcs
│   │   │   ├── d_rqznrr
│   │   │   │   └── f_105075_shslmlt.spg
│   │   │   ├── f_108476_slw

In [99]:
import unittest

class TestNotebook(unittest.TestCase):
    a = Node("d_a")
    b = Node("d_b")
    c = Node("f_5_c")
    d = Node("f_15_d")
    e = Node("f_100000_e")

    def setUp(self):
        global root, pwd
        root = Node("/")
        pwd = root
        self.a = Node("d_a")
        self.b = Node("d_b")
        self.c = Node("f_5_c")
        self.d = Node("f_15_d")
        self.e = Node("f_100000_e")
        root.children = [self.a, self.b]
        self.c.parent = self.b
        self.d.parent = self.b
        self.e.parent = self.a
    
    def test_execute_cd(self):
        global root, pwd
        self.assertEqual(root, pwd)
        execute_cd("$ cd a")
        self.assertEqual(pwd, self.a)

    def test_is_file_line(self):
        self.assertTrue(is_file_line("123 a"))
        self.assertTrue(is_file_line("1 b"))
        self.assertFalse(is_file_line("$ ls"))
        self.assertFalse(is_file_line("dir abc"))
    
    def test_get_node_size(self):
        global root
        self.assertEqual(20, get_node_size(self.b))
        self.assertEqual(100000, get_node_size(self.a))
        self.assertEqual(100020, get_node_size(root))

unittest.main(argv=[''], verbosity=2, exit=False)

test_execute_cd (__main__.TestNotebook) ... ok
test_get_node_size (__main__.TestNotebook) ... ok
test_is_file_line (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK
